## Data Manipulation and Storage Part I

This notebook is associated with the lesson titled **Data Manipulation and Storage Part II** in the Data Analysis and Storage Module. In this notebook we perform the following tasks:

    - Set up our environment so we can load pickle files
    - Test out components from the OffensiveTeamTable
    - Execute the OffensiveTeamTable and DefensiveTeamTable

In [ ]:
import os
import pickle
import sys
import pandas as pd
from tqdm import tqdm

from os.path import expanduser

# add path to directory that contains web.py to system
sys.path.insert(0, os.path.join(expanduser("~"), "teachdfs"))

from data import OffenseTeamTable, DefenseTeamTable
from config import CACHE_DIRECTORY
from maps import team_map_inv


In [ ]:
# cycle through seasons
season = "2018"

# path to pickle data
scores_path = os.path.join(CACHE_DIRECTORY, f"{season}_box.pkl")

with open(scores_path, "rb") as input_file:
    all_scores = pickle.load(input_file)
    

## Test Components From OffenseTeamTable

In [ ]:
def team_records_from_boxscore(fbs):
    """ Takes a boxscore, generates a dataframe such that the rows correspond to teams and the columns contain 
    stats related to team offensive performance. Returns the dataframe. """
    # Row 1
    home_stats_df = fbs.all_team_stats['home_stat'].copy()
    home_stats_df['team'] = team_map_inv[fbs.scorebox['home_team']]
    home_stats_df['date'] = pd.Timestamp(fbs.scorebox['date'])
    home_stats_df['opp'] = team_map_inv[fbs.scorebox['away_team']]
    # Row 2
    vis_stats_df = fbs.all_team_stats['vis_stat'].copy()
    vis_stats_df['team'] = team_map_inv[fbs.scorebox['away_team']]
    vis_stats_df['date'] = pd.Timestamp(fbs.scorebox['date'])
    vis_stats_df['opp'] = team_map_inv[fbs.scorebox['home_team']]
    table = pd.concat([home_stats_df, vis_stats_df], axis=1).T
    return table

In [ ]:
team_records_from_boxscore(all_scores[0])

In [ ]:
team_records = []
for fbs in tqdm(all_scores):
    team_records.append(team_records_from_boxscore(fbs))

In [ ]:
def build_team_table(team_table):
    """ Takes a dataframe of home and away team stats. Cleans up time-related columns and splits up compound
    columns. Returns the new dataframe. """

    team_table["Time of Possession"] = team_table["Time of Possession"].apply(
        lambda x: float(x.split(":")[0]) + float(x.split(":")[1]) / 60
    )
    t1 = team_table["Cmp-Att-Yd-TD-INT"].apply(
        lambda x: pd.Series(dict(zip(["pass_cmp", "pass_att", "pass_yd", "pass_td", "pass_int"], x.split("-"))))
    )
    t2 = team_table["Fourth Down Conv."].apply(
        lambda x: pd.Series(dict(zip(["fouth_conv_succ", "fouth_conv_att"],x.split("-"))))
    )
    t3 = team_table["Fumbles-Lost"].apply(
        lambda x: pd.Series(dict(zip(["fumbles", "fumbles_lost"], x.split("-"))))
    )
    t4 = team_table["Penalties-Yards"].apply(
        lambda x: pd.Series(dict(zip(["penalty_count", "penalty_yds"],x.split("-"))))
    )
    t5 = team_table["Rush-Yds-TDs"].apply(
        lambda x: pd.Series(dict(zip(["rush_att", "rush_yds", "rush_tds"], x.split("-"))))
    )
    t6 = team_table["Sacked-Yards"].apply(
        lambda x: pd.Series(dict(zip(["sacks_allowed", "sacks_allowed_yds"], x.split("-"))))
    )
    t7 = team_table["Third Down Conv."].apply(
        lambda x: pd.Series(dict(zip(["third_conv_succ", "third_conv_att"], x.split("-"))))
    )
    t8 = team_table[
        ["First Downs", "Net Pass Yards", "Time of Possession", 'Total Yards', 'Turnovers', 'team', 'date', 'opp']
    ]
    
    return pd.concat([t1, t2, t3, t4, t5, t6, t7, t8], axis=1)

In [ ]:
table_df = build_team_table(pd.concat(team_records))

In [ ]:
table_df

## Test OffenseTeamTable

In [ ]:
# cycle through seasons
for season in {"2018", "2019", "2020"}:

    # path to pickle data
    scores_path = os.path.join(CACHE_DIRECTORY, f"{season}_box.pkl")

    with open(scores_path, "rb") as input_file:
        all_scores = pickle.load(input_file)
    
    # process and cache OffenseTeamTable
    OffenseTeamTable(season=season, refresh=True, boxscores=all_scores)

In [ ]:
# inspect cached data
ott_2018 = OffenseTeamTable(season=2018)
ott_2018.table

## Test DefensiveTeamTable

In [ ]:
# cycle through seasons
for season in {"2018", "2019", "2020"}:

    # path to pickle data
    scores_path = os.path.join(CACHE_DIRECTORY, f"{season}_box.pkl")

    with open(scores_path, "rb") as input_file:
        all_scores = pickle.load(input_file)
    
    # process and cache OffenseTeamTable
    DefenseTeamTable(season=season, refresh=True, boxscores=all_scores)

In [ ]:
# inspect cached data
dtt_2018 = DefenseTeamTable(season=2018)
dtt_2018.table